# Set up

In [ ]:
save_dir = None
checkpoint_dir = None
keyframes_dir = None

In [ ]:
!git clone https://github.com/xinyu1205/recognize-anything.git
%pip install timm transformers fairscale pycocoevalcap

# change the working directory to the cloned repo
%cd recognize-anything

fatal: destination path 'recognize-anything' already exists and is not an empty directory.
Note: you may need to restart the kernel to use updated packages.
/home/heigatvu/my-project/competition/pipeline-hcm-ai/notebooks/data_extraction/metadata/recognize-anything


/home/heigatvu/my-project/competition/pipeline-hcm-ai/venv/lib/python3.9/site-packages/IPython/core/magics/osm.py:417: UserWarning: using dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [ ]:
import os

dir_path = os.getcwd() # Get the current directory path
parent_path = os.path.dirname(dir_path)

if save_dir is None:
    save_dir = f"{parent_path}/tag_output"
if checkpoint_dir is None:
    checkpoint_dir = f"{parent_path}/pretrained"
if keyframes_dir is None:
    keyframes_dir = os.path.join(os.path.dirname(parent_path), 'transnet', 'keyframes')


In [ ]:
import os
import glob
import json
import torch
import numpy as np
from PIL import Image
from ram.models import ram_plus
from ram import get_transform
from ram import inference_ram as inference
from tqdm import tqdm
import subprocess
from concurrent.futures import ThreadPoolExecutor

/home/heigatvu/my-project/competition/pipeline-hcm-ai/venv/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Download CheckPoint

In [ ]:
def download_checkpoints(model, checkpoint_dir):
    print('You selected', model)
    print(f'Checkpoint directory: {checkpoint_dir}')
    if not os.path.exists(checkpoint_dir):
        os.makedirs(checkpoint_dir)
    print(checkpoint_dir)

    if model == "RAM++":
        ram_weights_path = os.path.join(checkpoint_dir, 'ram_plus_swin_large_14m.pth')
        if not os.path.exists(ram_weights_path):
            url = "https://huggingface.co/xinyu1205/recognize-anything-plus-model/resolve/main/ram_plus_swin_large_14m.pth"
            subprocess.run(['wget', url, '-O', os.path.join(checkpoint_dir, 'ram_plus_swin_large_14m.pth')], check=True)
            print(f"RAM weights downloaded to {ram_weights_path}")
        else:
            print("RAM weights already downloaded!")

model = "RAM++"
download_checkpoints(model, checkpoint_dir)
print(model, 'weights are downloaded!')

You selected RAM++
Checkpoint directory: /home/heigatvu/my-project/competition/pipeline-hcm-ai/notebooks/data_extraction/metadata/pretrained
/home/heigatvu/my-project/competition/pipeline-hcm-ai/notebooks/data_extraction/metadata/pretrained
RAM weights already downloaded!
RAM++ weights are downloaded!


# Parse data path

In [ ]:
def parse_keyframe_info(keyframes_dir):
    all_keyframe_paths = {}
    for part in sorted(os.listdir(keyframes_dir)):
        data_part_path = f'{keyframes_dir}/{part}'
        data_part = part.split('/')[-1]
        all_keyframe_paths[data_part] = []
        image_path = sorted(glob.glob(f'{data_part_path}/*.jpg'))
        all_keyframe_paths[data_part] = image_path
    return all_keyframe_paths

all_data = parse_keyframe_info(keyframes_dir)
print(all_data)

{'L01_V001': ['/home/heigatvu/my-project/competition/pipeline-hcm-ai/notebooks/data_extraction/transnet/keyframes/L01_V001/001.jpg', '/home/heigatvu/my-project/competition/pipeline-hcm-ai/notebooks/data_extraction/transnet/keyframes/L01_V001/002.jpg', '/home/heigatvu/my-project/competition/pipeline-hcm-ai/notebooks/data_extraction/transnet/keyframes/L01_V001/003.jpg', '/home/heigatvu/my-project/competition/pipeline-hcm-ai/notebooks/data_extraction/transnet/keyframes/L01_V001/004.jpg', '/home/heigatvu/my-project/competition/pipeline-hcm-ai/notebooks/data_extraction/transnet/keyframes/L01_V001/005.jpg', '/home/heigatvu/my-project/competition/pipeline-hcm-ai/notebooks/data_extraction/transnet/keyframes/L01_V001/006.jpg', '/home/heigatvu/my-project/competition/pipeline-hcm-ai/notebooks/data_extraction/transnet/keyframes/L01_V001/007.jpg', '/home/heigatvu/my-project/competition/pipeline-hcm-ai/notebooks/data_extraction/transnet/keyframes/L01_V001/008.jpg', '/home/heigatvu/my-project/competi

# Inference

In [ ]:
# Configuration
PRETRAINED_MODEL_PATH = f'{checkpoint_dir}/ram_plus_swin_large_14m.pth'
IMAGE_SIZE = 384

def main():
    def create_directory(path):
        """Create a directory if it does not exist."""
        if not os.path.exists(path):
            os.makedirs(path)
    
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    transform = get_transform(image_size=IMAGE_SIZE)

    # Load model
    model = ram_plus(pretrained=PRETRAINED_MODEL_PATH, image_size=IMAGE_SIZE, vit='swin_l')
    model.eval()
    model = model.to(device)

    # Parse keyframe info
    all_keyframe_paths = parse_keyframe_info(keyframes_dir)


    # Process each keyframe
    for video_id, image_paths in tqdm(all_keyframe_paths.items()):
        # Initialize output dictionary
        output_data = {}
        for image_path in tqdm(image_paths, desc=f"Processing videos in {video_id}", leave=False):
            try:
                image = transform(Image.open(image_path)).unsqueeze(0).to(device)
                res = inference(image, model)
                tags = res[0].split(' | ')
                output_data[os.path.basename(image_path)] = tags
            except Exception as e:
                print(f"Error processing {image_path}: {str(e)}")   
        # Save output JSON
        json_path = os.path.join(
            save_dir, f'{video_id}.json')
        create_directory(os.path.dirname(json_path))
        with open(json_path, 'w') as f:
            json.dump(output_data, f)

In [ ]:

if __name__ == "__main__":
    main()

--------------
/home/heigatvu/my-project/competition/pipeline-hcm-ai/notebooks/data_extraction/metadata/pretrained/ram_plus_swin_large_14m.pth
--------------
load checkpoint from /home/heigatvu/my-project/competition/pipeline-hcm-ai/notebooks/data_extraction/metadata/pretrained/ram_plus_swin_large_14m.pth
vit: swin_l


100%|██████████| 4/4 [03:02<00:00, 45.74s/it]


# Reformat tag (optional)

In [ ]:
import re
from collections import defaultdict

def rename_files(folder_path):
    # Regular expressions for matching file names
    pattern_extra = re.compile(r'(L\d+_V\d+)_extra_tagging\.json')
    pattern_normal = re.compile(r'(L\d+_V\d+)_tagging\.json')

    # Counters for summary
    summary = defaultdict(int)

    # Iterate through all files in the folder
    for filename in os.listdir(folder_path):
        file_path = os.path.join(folder_path, filename)
        
        # Check if it's a file (not a directory)
        if os.path.isfile(file_path):
            # Try to match with the 'extra' pattern
            match_extra = pattern_extra.match(filename)
            if match_extra:
                new_filename = f"{match_extra.group(1)}_extra.json"
                new_file_path = os.path.join(folder_path, new_filename)
                try:
                    os.rename(file_path, new_file_path)
                    summary['renamed_extra'] += 1
                except OSError as e:
                    print(f"Error renaming {filename}: {e}")
                    summary['errors'] += 1
            
            # Try to match with the 'normal' pattern
            elif pattern_normal.match(filename):
                match_normal = pattern_normal.match(filename)
                new_filename = f"{match_normal.group(1)}.json"
                new_file_path = os.path.join(folder_path, new_filename)
                try:
                    os.rename(file_path, new_file_path)
                    summary['renamed_normal'] += 1
                except OSError as e:
                    print(f"Error renaming {filename}: {e}")
                    summary['errors'] += 1
            
            else:
                summary['skipped'] += 1

    return summary

In [ ]:
# Specify the folder path where your JSON files are located
folder_path = '/home/heigatvu/my-project/competition/pipeline-hcm-ai/notebooks/data_extraction/metadata/tag_output'  # Current directory, change this if your JSON files are in a different folder

# Run the renaming function
print("Starting file renaming process...")
results = rename_files(folder_path)

Starting file renaming process...


In [ ]:
rm -rf ../recognize-anything

In [ ]:
rm -rf ../pretrained